In [6]:
import pandas as pd
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_community.llms import GigaChat
from pydantic import BaseModel, Field
from typing import Optional
import json
import re
import requests
from typing import Dict, Any, List
import base64
from clearml import Task, Logger
import os
import nltk
from nltk.tokenize import word_tokenize
import re

In [19]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [43]:
def count_words_oge(text):
    """
    Подсчет слов по правилам ОГЭ:
    - Учитываются последовательности символов без пробелов
    - Инициалы с фамилией считаются одним словом
    - Цифры и другие не-буквенные символы не учитываются
    """
    if not text or not isinstance(text, str):
        return 0
    
    # Предварительная обработка для объединения инициалов с фамилией
    text = re.sub(r'([А-ЯЁ]\.\s*[А-ЯЁ]\.\s*[А-ЯЯёЁ]+)', 'ФИО', text)
    text = re.sub(r'([А-ЯЁ]\.\s*[А-ЯЯёЁ]+)', 'ФИО', text)
    
    # Токенизация с учетом специфики русского языка
    words = word_tokenize(text, language='russian')
    
    # Фильтрация: оставляем только слова, содержащие буквы
    valid_words = []
    for word in words:
        # Убираем знаки препинания по краям
        clean_word = word.strip('.,!?;:"()[]{}«»-–—')
        # Если слово содержит хотя бы одну букву - учитываем
        if clean_word and any(c.isalpha() for c in clean_word):
            valid_words.append(clean_word)
    
    return len(valid_words)

In [ ]:
# Определяем структуру вывода с помощью Pydantic
class EssayEvaluation(BaseModel):
    H1: int = Field(description="Балл по критерию K1", ge=0, le=1, default=0)
    H1_explanation: str = Field(description="Обоснование оценки по критерию K1", default="")
    H2: int = Field(description="Балл по критерию K2", ge=0, le=3, default = 0)
    H2_explanation: str = Field(description="Обоснование оценки по критерию K2", default="")
    H3: int = Field(description="Балл по критерию K3", ge=0, le=2, default = 0)
    H3_explanation: str = Field(description="Обоснование оценки по критерию K3", default="")
    H4: int = Field(description="Балл по критерию K4", ge=0, le=1, default = 0)
    H4_explanation: str = Field(description="Обоснование оценки по критерию K4", default="")

# Настройка GigaChat 
llm = GigaChat(
    credentials="", 
    verify_ssl_certs=False,
    model='GigaChat-2',
    temperature=0.1
)

# Создаем парсер вывода
parser = PydanticOutputParser(pydantic_object=EssayEvaluation)


In [46]:

# Промпт для типа 1 (13.1 - лингвистическое сочинение)
# prompt_type1 = ChatPromptTemplate.from_messages([
#     ("system", """Ты - эксперт по проверке сочинений ОГЭ по русскому языку. 
# Оцени сочинение по критериям К1-К4 для задания типа 13.1 (лингвистическое сочинение).

# КРИТЕРИИ ОЦЕНКИ для типа 1:
# К1 (0-1 балл): Наличие обоснованного ответа на лингвистический вопрос. Рассуждение на теоретическом уровне. 1 балл: Ученик верно понял и прокомментировал высказывание. 0 баллов: Тезис отсутствует или неверен.
# К2 (0-3 балла): Приведение двух примеров-иллюстраций из опорного текста, соответствующих теме.3 балла: Приведено 2 примера из текста, оба иллюстрируют тезис. 2 балла: 2 примера, но один из них слабо или неполно иллюстрирует тезис. 1 балл: Приведен 1 пример, который иллюстрирует тезис. 0 баллов: Примеры отсутствуют или не являются аргументами.
# К3 (0-2 балла): Логичность речи - смысловая цельность, связность, последовательность изложения. 0 ошибок = 2 балла, 1 ошибка = 1 балл, >1 ошибки = 0 баллов.
# К4 (0-1 балл): Композиционная стройность - трехчастная композиция, завершенность. ≤1 ошибка = 1 балл, иначе 0.
# По каждому критерию ты должен дать пояснения к оценке, которую ты поставил.
# Текст задания: {task_text}
# Тип сочинения: 1 (13.1)

# Верни ответ строго в указанном JSON-формате."""),
#     ("human", "Текст сочинения: {essay_text}\n\n{format_instructions}")
# ]).partial(format_instructions=parser.get_format_instructions())

# Промпт для типа 2 (13.2 - литературно-тематическое сочинение)
# promt2 = """Ты - эксперт по проверке сочинений ОГЭ по русскому языку. 
# Оцени сочинение по критериям К1-К4 для задания типа 13.2 (литературно-тематическое сочинение).

# КРИТЕРИИ ОЦЕНКИ для типа 2:
# К1 (0-1 балл): Понимание смысла фрагмента текста - верное объяснение содержания фрагмента. 1 балл: Ученик дал  дал верное объяснение содержания фрагмента текста. Ошибок в интерпретации нет. 0 баллов: если экзаменуемый дал в целом верное объяснение содержания фрагмента текста, но в интерпретации допустил одну ошибку или более, или экзаменуемый дал неверное объяснение содержания фрагмента текста, или объяснение содержания фрагмента текста в работе экзаменуемого отсутствует
# К2 (0-3 балла): Приведение двух примеров-иллюстраций из опорного текста, иллюстрирующих понимание фрагмента. 3 балла: Приведено 2 примера из текста, оба поясняют смысл фрагмента. 2 балла: один пример из прочитанного текста, соответствующий объяснению содержания данного фрагмента. 1 балл: пример(ы) не из прочитанного текста. 0 баллов: Примеры отсутствуют или не являются аргументами или или экзаменуемый привёл в качестве примера данную в задании цитату или её часть 
# К3 (0-2 балла): Логичность речи - смысловая цельность, связность, последовательность изложения. 2 балла: Логические ошибки отсутствуют, 1 балл: Допущена одна логическая ошибка, 0 баллов: Допущено две логические ошибки или более
# К4 (0-1 балл): Композиционная стройность - трехчастная композиция, завершенность. 1 балл: Работа характеризуется трёхчастной композицией, ошибки в построении текста отсутствуют, 0 баллов: В работе нарушена трёхчастная композиция или в построении текста допущена одна ошибка или более
# По каждому критерию ты должен дать пояснения к оценке, которую ты поставил.
# По всем критериям выстави 0 баллов, если сочинение представляет собой полностью переписанный
# или пересказанный текст или если сочинение написано на основе фрагмента текста, отличного
# от фрагмента текста в задании 13.2 выполняемого варианта или если в сочинении менее 70 слов
# Текст задания: {task_text}
# Тип сочинения: 2 (13.2)

# Верни ответ строго в указанном JSON-формате."""
promt2 = """Ты - эксперт по проверке сочинений ОГЭ по русскому языку. 
Оцени сочинение по критериям К1-К4 для задания типа 13.2 (литературно-тематическое сочинение).

КРИТЕРИИ ОЦЕНКИ для типа 2:
К1 (0-1 балл): Понимание смысла фрагмента текста - верное объяснение содержания фрагмента. 1 балл: Ученик дал ясное и корректное объяснение смысла фрагмента. 0 баллов: Объяснение отсутствует или неверно.
К2 (0-3 балла): Приведение двух примеров-иллюстраций из опорного текста, иллюстрирующих понимание фрагмента. 3 балла: Приведено 2 примера из текста, оба поясняют смысл фрагмента. 2 балла: 2 примера, но один из них слабо поясняет смысл. 1 балл: Приведен 1 пример, который поясняет смысл фрагмента. 0 баллов: Примеры отсутствуют или не являются аргументами.
К3 (0-2 балла): Логичность речи - смысловая цельность, связность, последовательность изложения. 0 ошибок = 2 балла, 1 ошибка = 1 балл, >1 ошибки = 0 баллов.
К4 (0-1 балл): Композиционная стройность - трехчастная композиция, завершенность. ≤1 ошибка = 1 балл, иначе 0.
По каждому критерию ты должен дать пояснения к оценке, которую ты поставил.
Если в сочинении менее 70 слов, выстави 0 баллов по всем критериям
Текст задания: {task_text}
Тип сочинения: 2 (13.2)

Верни ответ строго в указанном JSON-формате."""
prompt_type2 = ChatPromptTemplate.from_messages([
    ("system", promt2),
    ("human", "Текст сочинения: {essay_text}\n\n{format_instructions}")
]).partial(format_instructions=parser.get_format_instructions())

# Промпт для типа 3 (13.3 - морально-нравственное сочинение)
# promt3 = """Ты - эксперт по проверке сочинений ОГЭ по русскому языку. 
# Оцени сочинение по критериям К1-К4 для задания типа 13.3 (морально-нравственное сочинение).

# КРИТЕРИИ ОЦЕНКИ для типа 3:
# К1 (0-1 балл): Наличие обоснованного ответа на поставленный вопрос. 1 балл: Экзаменуемый дал обоснованный ответ на вопрос, сформулированный в теме сочинения. 0 баллов: Экзаменуемый дал необоснованный ответ на вопрос, сформулированный в теме сочинения, или экзаменуемый не ответил на вопрос, сформулированный в теме сочинения
# К2 (0-3 балла): Приведение одного примера из опорного текста и одного примера из личного опыта/литературы. 3 балла: Экзаменуемый привёл два примера: один пример из прочитанного текста, а другой – из жизненного опыта, или экзаменуемый привёл два примера из прочитанного текста. 2 балла: Экзаменуемый привёл один пример из прочитанного текста. 1 балл: ПЭкзаменуемый привёл пример(ы) из жизненного опыта. 0 баллов: Примеры отсутствуют или не являются аргументами. (Не учитываются примеры, источниками которых являются комикс, аниме, манга, фанфик, графический роман, компьютерная игра)
# К3 (0-2 балла): Логичность речи - смысловая цельность, связность, последовательность изложения. 2 балла: Логические ошибки отсутствуют, 1 балл: Допущена одна логическая ошибка, 0 баллов: Допущено две логические ошибки или более
# К4 (0-1 балл): Композиционная стройность - трехчастная композиция, завершенность. 1 балл: Работа характеризуется трёхчастной композицией, ошибки в построении текста отсутствуют, 0 баллов: В работе нарушена трёхчастная композиция или в построении текста допущена одна ошибка или более
# По каждому критерию ты должен дать пояснения к оценке, которую ты поставил.
# Выстави 0 баллов по всем критериям, если сочинение представляет собой полностью переписанный
# или пересказанный текст, если сочинение написано на основе на основе вопроса, отличного
# от вопроса в задании 13.3 выполняемого варианта, если в сочинении менее 70 слов
# Текст задания: {task_text}
# Тип сочинения: 3 (13.3)

# Верни ответ строго в указанном JSON-формате."""
promt3 = """Ты - эксперт по проверке сочинений ОГЭ по русскому языку. 
Оцени сочинение по критериям К1-К4 для задания типа 13.3 (морально-нравственное сочинение).

КРИТЕРИИ ОЦЕНКИ для типа 3:
К1 (0-1 балл): Наличие обоснованного ответа на поставленный вопрос. 1 балл: Есть как минимум простое определение понятия И ответ на вопрос темы (тезис). 0 баллов: Определения или ответа на вопрос нет.
К2 (0-3 балла): Приведение одного примера из опорного текста и одного примера из личного опыта/литературы. 3 балла: Приведено 2 примера (1 из текста + 1 из жизни), оба иллюстрируют тезис. 2 балла: 2 примера, но один из них слабо иллюстрирует тезис. 1 балл: Приведен 1 пример (из текста или жизни), который иллюстрирует тезис. 0 баллов: Примеры отсутствуют или не являются аргументами.
К3 (0-2 балла): Логичность речи - смысловая цельность, связность, последовательность изложения. 0 ошибок = 2 балла, 1 ошибка = 1 балл, >1 ошибки = 0 баллов.
К4 (0-1 балл): Композиционная стройность - трехчастная композиция, завершенность. ≤1 ошибка = 1 балл, иначе 0.
По каждому критерию ты должен дать пояснения к оценке, которую ты поставил.
Если в сочинении менее 70 слов, выстави 0 баллов по всем критериям
Текст задания: {task_text}
Тип сочинения: 3 (13.3)

Верни ответ строго в указанном JSON-формате."""
prompt_type3 = ChatPromptTemplate.from_messages([
    ("system", promt3),
    ("human", "Текст сочинения: {essay_text}\n\n{format_instructions}")
]).partial(format_instructions=parser.get_format_instructions())


In [47]:

# Безопасный парсер для обработки ошибок
class SafePydanticOutputParser(PydanticOutputParser):
    def parse(self, text: str):
        try:
            # Пытаемся найти JSON в тексте (модель может добавлять пояснения)
            json_match = re.search(r'\{.*\}', text, re.DOTALL)
            if json_match:
                json_text = json_match.group()
                data = json.loads(json_text)
                
                # Обрабатываем None значения
                for field in ['H1', 'H2', 'H3', 'H4']:
                    if field in data and data[field] is None:
                        data[field] = 0
                for field in ['H1_explanation', 'H2_explanation', 'H3_explanation', 'H4_explanation']:
                    if field in data and data[field] is None:
                        data[field] = "Обоснование не предоставлено"
                
                return self.pydantic_object.parse_obj(data)
            else:
                # Если JSON не найден, создаем объект с значениями по умолчанию
                return self.pydantic_object.parse_obj({
                    'H1': 0, 'H1_explanation': 'Не удалось извлечь оценку',
                    'H2': 0, 'H2_explanation': 'Не удалось извлечь оценку',
                    'H3': 0, 'H3_explanation': 'Не удалось извлечить оценку',
                    'H4': 0, 'H4_explanation': 'Не удалось извлечь оценку'
                })
        except Exception as e:
            print(f"Ошибка парсинга: {str(e)}")
            return self.pydantic_object.parse_obj({
                'H1': 0, 'H1_explanation': f'Ошибка парсинга: {str(e)}',
                'H2': 0, 'H2_explanation': f'Ошибка парсинга: {str(e)}',
                'H3': 0, 'H3_explanation': f'Ошибка парсинга: {str(e)}',
                'H4': 0, 'H4_explanation': f'Ошибка парсинга: {str(e)}'
            })

# Создаем безопасный парсер
safe_parser = SafePydanticOutputParser(pydantic_object=EssayEvaluation)

# Создаем цепочки для каждого типа сочинения
# chain1 = LLMChain(llm=llm, prompt=prompt_type1, output_parser=safe_parser)
chain2 = LLMChain(llm=llm, prompt=prompt_type2, output_parser=safe_parser)
chain3 = LLMChain(llm=llm, prompt=prompt_type3, output_parser=safe_parser)


In [48]:
def evaluate_essays(essays_df, reference_texts_dict, output_path="submission.csv"):
    """Основная функция для оценки сочинений"""
    results = []
    
    print(f"Начинаем обработку {len(essays_df)} сочинений...")
    
    for idx, row in essays_df.iterrows():
        try:
            essay_id = row['essay_id']
            essay_type = row['essay_type']
            essay_text = row['essay_text']
            reference_text_id = row['reference_text_id']
            
            # 🔴 ПРОПУСК СОЧИНЕНИЙ ТИПА 13.1
            if essay_type == 1:
                print(f"Пропуск сочинения {essay_id}: тип 13.1 не оценивается")
                results.append({
                    "essay_id": essay_id,
                    "H1": 0,
                    "H1_explanation": "Тип сочинения 13.1 (лингвистическое) не оценивается",
                    "H2": 0,
                    "H2_explanation": "Тип сочинения 13.1 (лингвистическое) не оценивается",
                    "H3": 0,
                    "H3_explanation": "Тип сочинения 13.1 (лингвистическое) не оценивается",
                    "H4": 0,
                    "H4_explanation": "Тип сочинения 13.1 (лингвистическое) не оценивается"
                })
                continue
            
            # 🔴 ПРОВЕРКА ОБЪЕМА СОЧИНЕНИЯ
            word_count = count_words_oge(essay_text)
            print(f"Сочинение {essay_id}: {word_count} слов")
            
            if word_count < 70:
                print(f"Сочинение {essay_id} пропущено: недостаточный объем ({word_count} < 70 слов)")
                results.append({
                    "essay_id": essay_id,
                    "H1": 0,
                    "H1_explanation": f"Недостаточный объем сочинения: {word_count} слов при требуемых 70",
                    "H2": 0,
                    "H2_explanation": f"Недостаточный объем сочинения: {word_count} слов при требуемых 70",
                    "H3": 0,
                    "H3_explanation": f"Недостаточный объем сочинения: {word_count} слов при требуемых 70",
                    "H4": 0,
                    "H4_explanation": f"Недостаточный объем сочинения: {word_count} слов при требуемых 70"
                })
                continue
            
            # Получаем reference текст
            if reference_text_id not in reference_texts_dict:
                print(f"Предупреждение: reference_text_id {reference_text_id} не найден")
                reference_data = {
                    'reference_text': 'Текст не найден',
                    'task': row.get('task_text', 'Задание не указано')
                }
            else:
                reference_data = reference_texts_dict[reference_text_id]
            
            # Получаем предсказание в зависимости от типа сочинения
            if essay_type == 2:
                result = chain2.invoke({
                    "essay_text": essay_text, 
                    "task_text": reference_data['task']
                })
            elif essay_type == 3:
                result = chain3.invoke({
                    "essay_text": essay_text, 
                    "task_text": reference_data['task']
                })
            else:
                print(f"Неизвестный тип сочинения: {essay_type}")
                continue
                
            evaluation = result["text"]
            
            results.append({
                "essay_id": essay_id,
                "H1": evaluation.H1,
                "H1_explanation": evaluation.H1_explanation,
                "H2": evaluation.H2,
                "H2_explanation": evaluation.H2_explanation,
                "H3": evaluation.H3,
                "H3_explanation": evaluation.H3_explanation,
                "H4": evaluation.H4,
                "H4_explanation": evaluation.H4_explanation
            })
            
            print(f"Обработано сочинение {idx + 1}/{len(essays_df)} (ID: {essay_id})")
            
        except Exception as e:
            print(f"Ошибка при обработке essay_id {row['essay_id']}: {str(e)}")
            results.append({
                "essay_id": row["essay_id"],
                "H1": 0,
                "H1_explanation": f"Ошибка обработки: {str(e)}",
                "H2": 0,
                "H2_explanation": f"Ошибка обработки: {str(e)}",
                "H3": 0,
                "H3_explanation": f"Ошибка обработки: {str(e)}",
                "H4": 0,
                "H4_explanation": f"Ошибка обработки: {str(e)}"
            })
    
    # Создаем DataFrame с результатами
    results_df = pd.DataFrame(results)
    
    # Сохраняем результаты
    results_df.to_csv(output_path, index=False)
    print(f"Результаты сохранены в {output_path}")
    
    return results_df

In [49]:
def test_word_count():
    """Тестирование функции подсчета слов"""
    test_texts = [
        "Привет, как дела?",
        "М.Ю. Лермонтов - великий поэт.",  # 4 слова (ФИО считается как одно)
        "Я прочитал 5 книг за лето.",      # 5 слов (цифра не считается)
        "Из-за дождя мы остались дома.",   # 5 слов ("из-за" - одно слово)
        "Несмотря на погоду, мы пошли гулять."  # 6 слов ("несмотря на" - два слова)
    ]
    
    for text in test_texts:
        count = count_words_oge(text)
        print(f"Текст: '{text}'")
        print(f"Слов: {count}\n")

In [50]:
def prepare_validation_data(results_df, essays_df) -> Dict[str, Any]:
    """
    Подготавливает данные для валидации в формате JSON
    ИСКЛЮЧАЕТ сочинения типа 1 (13.1)
    
    Args:
        results_df: DataFrame с результатами оценок
        essays_df: исходный DataFrame с информацией о типах сочинений
        
    Returns:
        Dict[str, Any]: Данные в формате для отправки на сервер
    """
    # Создаем список строк с оценками
    rows = []
    
    for idx, row in results_df.iterrows():
        essay_id = str(row['essay_id'])
        
        # 🔴 ПРОВЕРЯЕМ ТИП СОЧИНЕНИЯ В ИСХОДНЫХ ДАННЫХ
        original_essay = essays_df[essays_df['essay_id'] == int(essay_id)]
        
        if len(original_essay) > 0:
            essay_type = original_essay.iloc[0]['essay_type']
            
            # 🔴 ПРОПУСК СОЧИНЕНИЙ ТИПА 1
            if essay_type == 1:
                print(f"Пропуск сочинения {essay_id} при подготовке JSON: тип 13.1 не отправляется на валидацию")
                continue
        
        row_data = {
            "essay_id": essay_id,
            "K1": int(row['H1']),  # H1 соответствует K1
            "K2": int(row['H2']),  # H2 соответствует K2  
            "K3": int(row['H3']),  # H3 соответствует K3
            "K4": int(row['H4'])   # H4 соответствует K4
        }
        rows.append(row_data)
    
    print(f"📊 На валидацию отправляется {len(rows)} сочинений (исключены тип 13.1)")
    
    # Формируем полную структуру JSON
    validation_data = {
        "DataSet": {
            "Rows": rows
        },
        "Variables2": {
            "UserID": USER_ID  # Используем глобальную переменную USER_ID
        }
    }
    
    return validation_data

In [51]:

def setup_clearml_task(task_name="Essay Evaluation Pipeline", project_name="OGE_Essay_Scoring"):
    """
    Инициализирует задачу ClearML и настраивает конфигурации
    
    Returns:
        Task: объект задачи ClearML
    """
    try:
        task = Task.init(
            project_name=project_name,
            task_name=task_name,
            task_type=Task.TaskTypes.training,
            reuse_last_task_id=False
        )
        
        # 🔧 Конфигурация промтов для LLM
        prompt_config = {
            "prompt_type1_description": "Лингвистическое сочинение (13.1)",
            "prompt_type2_description": "Литературно-тематическое сочинение (13.2)", 
            "prompt_type3_description": "Морально-нравственное сочинение (13.3)",
            "llm_model": "GigaChat-2",
            "temperature": 0.1,
            "output_parser": "PydanticOutputParser",
            "evaluation_criteria": "K1-K4 по спецификации ОГЭ"
        }
        
        # Конфигурация критериев оценки
        criteria_config = {
            "K1_range": "0-1 балл",
            "K2_range": "0-3 балла", 
            "K3_range": "0-2 балла",
            "K4_range": "0-1 балл",
            "K1_description": "Понимание и обоснование лингвистического вопроса",
            "K2_description": "Примеры-иллюстрации из текста",
            "K3_description": "Логичность и связность речи",
            "K4_description": "Композиционная стройность"
        }
        
        # Подключаем конфигурации к задаче
        task.connect(prompt_config, name="LLM_Prompts_Config")
        task.connect(criteria_config, name="Evaluation_Criteria_Config")
        
        print("✅ ClearML задача инициализирована с конфигурациями промтов")
        return task
        
    except Exception as e:
        print(f"❌ Ошибка инициализации ClearML: {str(e)}")
        return None
    
def log_clearml_metrics(task, results_df, validation_result, model_info=None):
    """
    Логирует метрики и результаты в ClearML с выделением score
    
    Args:
        task: объект задачи ClearML
        results_df: DataFrame с результатами оценок
        validation_result: результаты валидации от сервиса
        model_info: информация о модели (опционально)
    """
    if task is None:
        return
        
    try:
        # Логируем основные метрики
        logger = task.get_logger()
        
        # Средние оценки по критериям
        task.log_metrics({
            "avg_K1": results_df['H1'].mean(),
            "avg_K2": results_df['H2'].mean(),
            "avg_K3": results_df['H3'].mean(),
            "avg_K4": results_df['H4'].mean(),
            "total_essays": len(results_df)
        })
        
        # 🔥 ОСОБЕННО ВАЖНО: Логируем основной score из валидации
        if "error" not in validation_result:
            # Ищем score в различных возможных полях ответа
            score_value = None
            
            # Прямое поле score
            if "score" in validation_result:
                score_value = validation_result["score"]
            # Ищем вложенные поля с score
            elif "metrics" in validation_result and "score" in validation_result["metrics"]:
                score_value = validation_result["metrics"]["score"]
            # Ищем любые поля, содержащие "score" в названии
            else:
                for key, value in validation_result.items():
                    if "score" in key.lower() and isinstance(value, (int, float)):
                        score_value = value
                        break
            
            # Логируем найденный score
            if score_value is not None:
                task.log_metrics({"main_validation_score": score_value})
                print(f"✅ Основной score валидации: {score_value}")
            
            # Дополнительно логируем все числовые метрики
            for key, value in validation_result.items():
                if isinstance(value, (int, float)):
                    task.log_metrics({f"validation_{key}": value})
        
        # Логируем распределение оценок
        for criterion in ['H1', 'H2', 'H3', 'H4']:
            value_counts = results_df[criterion].value_counts().sort_index()
            for score, count in value_counts.items():
                logger.report_scalar(
                    title=f"Score Distribution {criterion}",
                    series="count",
                    value=count,
                    iteration=score
                )
        
        # Логируем информацию о модели
        if model_info:
            task.connect(model_info)
            
        print("✅ Метрики записаны в ClearML")
        
    except Exception as e:
        print(f"❌ Ошибка логирования в ClearML: {str(e)}")

def log_clearml_artifacts(task, file_paths):
    """
    Логирует артефакты (файлы) в ClearML
    
    Args:
        task: объект задачи ClearML
        file_paths: список путей к файлам для логирования
    """
    if task is None:
        return
        
    try:
        for file_path in file_paths:
            if os.path.exists(file_path):
                task.upload_artifact(
                    name=os.path.basename(file_path),
                    artifact_object=file_path
                )
                print(f"✅ Артефакт {file_path} загружен в ClearML")
            else:
                print(f"⚠️ Файл {file_path} не найден")
                
    except Exception as e:
        print(f"❌ Ошибка загрузки артефактов в ClearML: {str(e)}")
        
def save_model_artifacts(llm, chains, output_dir="model_artifacts"):
    """
    Сохраняет артефакты модели и промты для последующего использования
    
    Args:
        llm: объект языковой модели
        chains: словарь с цепочками
        output_dir: директория для сохранения
        
    Returns:
        dict: информация о модели для логирования
    """
    try:
        os.makedirs(output_dir, exist_ok=True)
        
        # Сохраняем тексты промтов
        prompt_texts = {
#             "prompt_type1": """
# Ты - эксперт по проверке сочинений ОГЭ по русскому языку. 
# Оцени сочинение по критериям К1-К4 для задания типа 13.1 (лингвистическое сочинение).

# КРИТЕРИИ ОЦЕНКИ для типа 1:
# К1 (0-1 балл): Наличие обоснованного ответа на лингвистический вопрос. Рассуждение на теоретическом уровне. 1 балл: Ученик верно понял и прокомментировал высказывание. 0 баллов: Тезис отсутствует или неверен.
# К2 (0-3 балла): Приведение двух примеров-иллюстраций из опорного текста, соответствующих теме.3 балла: Приведено 2 примера из текста, оба иллюстрируют тезис. 2 балла: 2 примера, но один из них слабо или неполно иллюстрирует тезис. 1 балл: Приведен 1 пример, который иллюстрирует тезис. 0 баллов: Примеры отсутствуют или не являются аргументами.
# К3 (0-2 балла): Логичность речи - смысловая цельность, связность, последовательность изложения. 0 ошибок = 2 балла, 1 ошибка = 1 балл, >1 ошибки = 0 баллов.
# К4 (0-1 балл): Композиционная стройность - трехчастная композиция, завершенность. ≤1 ошибка = 1 балл, иначе 0.
#             """,
            "prompt_type2": promt2,
            "prompt_type3": promt3
        }
        
        model_info = {
            "model_name": "GigaChat-2",
            "temperature": 0.1,
            "prompt_templates": {
                # "type1": "лингвистическое сочинение",
                "type2": "литературно-тематическое сочинение", 
                "type3": "морально-нравственное сочинение"
            },
            "output_parser": "PydanticOutputParser",
            "timestamp": pd.Timestamp.now().isoformat()
        }
        
        # Сохраняем информацию о модели
        with open(os.path.join(output_dir, "model_info.json"), "w", encoding="utf-8") as f:
            json.dump(model_info, f, ensure_ascii=False, indent=2)
        
        # Сохраняем тексты промтов
        with open(os.path.join(output_dir, "prompt_texts.json"), "w", encoding="utf-8") as f:
            json.dump(prompt_texts, f, ensure_ascii=False, indent=2)
            
        print(f"✅ Артефакты модели и промты сохранены в {output_dir}")
        return model_info
        
    except Exception as e:
        print(f"❌ Ошибка сохранения артефактов модели: {str(e)}")
        return {}
    
def log_validation_details(task, validation_result, validation_data):
    """
    Логирует детали валидации в ClearML
    
    Args:
        task: объект задачи ClearML
        validation_result: результаты валидации
        validation_data: отправленные данные
    """
    if task is None:
        return
        
    try:
        # Логируем структуру отправленных данных
        validation_info = {
            "submitted_essays_count": len(validation_data["DataSet"]["Rows"]),
            "user_id": validation_data["Variables2"]["UserID"],
            "validation_timestamp": pd.Timestamp.now().isoformat(),
            "score": validation_result["DataSet"]["Rows"][0]["Score"]
        }
        
        # Добавляем score, если он есть
        if "error" not in validation_result:
            score_value = None
            if "score" in validation_result:
                score_value = validation_result["DataSet"]["Rows"][0]["Score"]
            elif "metrics" in validation_result and "score" in validation_result["metrics"]:
                score_value = validation_result["metrics"]["score"]
            else:
                for key, value in validation_result.items():
                    if "score" in key.lower() and isinstance(value, (int, float)):
                        score_value = value
                        break
            
            if score_value is not None:
                validation_info["main_score"] = score_value
                # Выводим score в консоль для наглядности
                print(f"🎯 ОСНОВНОЙ SCORE ВАЛИДАЦИИ: {score_value}")
        
        task.connect(validation_info, name="Validation_Details")
        print("✅ Детали валидации записаны в ClearML")
        
    except Exception as e:
        print(f"❌ Ошибка логирования деталей валидации: {str(e)}")


In [52]:

# Основной блок выполнения
if __name__ == "__main__":
    # Загрузка данных
    essays_df = pd.read_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/project/essays.csv")  # Укажите правильный путь
    ref_text1 = pd.read_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/project/reference_text1.csv")
    ref_text2 = pd.read_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/project/reference_text2.csv")
    ref_text3 = pd.read_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/project/reference_text3.csv")
    ref_text4 = pd.read_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/project/reference_text4.csv")
    ref_text5 = pd.read_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/project/reference_text5.csv")
    ref_text6 = pd.read_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/project/reference_text6.csv")
    ref_text7 = pd.read_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/project/reference_text7.csv")
    ref_text8 = pd.read_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/project/reference_text8.csv")
    texts = [ref_text1, ref_text2, ref_text3, ref_text4, ref_text5, ref_text6, ref_text7, ref_text8]
    # Оценка сочинений
    reference_texts = {}
    for i in range(0,8):
        reference_texts[i+1] = texts[i]
    
#     manual_essay_df = pd.DataFrame({
#     'essay_id': [999],  # Уникальный ID
#     'essay_text': ["Что такое бескорыстность? Бескорыстность - это способность человека помочь, не ищя для себя выгоды, и не ждать при этом благодарности. "
#     "Бескорыстность красит человека и характиеризует его отношение к другим, готовность помочь без повода и вознаграждения. Докажу свои высказывания тезисами из текста. "
#     "В тексте мальчик ехал в поезде с книгой. Он поставил ее на открытое окно, отвлекся, и книга исчезла между двойными окначи вагона. Сосед-летчик, а потом и весь вагон, понимали, что маленький ребенок ничем не смжет их вознаградить, проявили бескорыстность и попытались достать книгу на протяжении всей дороги. "
#     "Даже после того, как мальчик вышел из поезда из-за прибытия на станцию, пассажиры вагона продолжили попытки, несмотря на то что ребенка,которому принаджедит книга, уже нет рядом. Многие на их месте уже давно бы бросили, но они упорно продолжали доставать книгу. Достав ее, сосед-летчик отослал книгу мальчику по почте, не потребовав денег или чего-то еще за свои старания. "
#     "В моей жизни была похожая ситуация. В один день, по дороге домой, я увидел горящий дом. На улице было много людей, успевших выбежать из того дома, и, кричавших, что внутри остались еще дети и раненные люди. И тут я заметил мужчину, смело шагающего по карнизу горящего дома. Он залез в окно и на протяжении двадцати минут детей и раненных и после убежал, ничего не сказав. Он не потребовал вознаграждения, благодарности или чего-то еще. Он рискнул своей жизнью ради других, проявив бескорыстную помощь. "
#     "В заключение хотелось бы сказать, что настоящие добрые люди - это те, кто способен на бескорыстную помощь."],
#     'essay_type': [3],  # 1, 2 или 3 в зависимости от типа задания
#     'reference_text_id': [6],  # ID опорного текста из ваших reference_text файлов
# })
    # results_df = evaluate_essays(manual_essay_df, reference_texts)
    results_df = evaluate_essays(essays_df, reference_texts)
    
    # Вывод статистики
    print("\nСтатистика оценок:")
print(f"Обработано сочинений: {len(results_df)}")
print(f"Средний балл К1: {results_df['H1'].mean():.2f}")
print(f"Средний балл К2: {results_df['H2'].mean():.2f}")
print(f"Средний балл К3: {results_df['H3'].mean():.2f}")
print(f"Средний балл К4: {results_df['H4'].mean():.2f}")

# Сохраняем полные объяснения в файл
with open("pred_expl.txt", "w", encoding="utf-8") as f:
    for idx, row in results_df.iterrows():
        f.write(f"Сочинение {row['essay_id']}:\n")
        f.write(f"К1 ({row['H1']} баллов): {row['H1_explanation']}\n")
        f.write(f"К2 ({row['H2']} баллов): {row['H2_explanation']}\n")
        f.write(f"К3 ({row['H3']} баллов): {row['H3_explanation']}\n")
        f.write(f"К4 ({row['H4']} баллов): {row['H4_explanation']}\n")
        f.write("-" * 50 + "\n")

USER_ID = "spbstu-ml-1"

print("\n🔄 Подготовка данных для валидации...")
print(f"🔑 Используется UserID: {USER_ID}")
    
    # Подготавливаем данные в правильном формате
validation_data = prepare_validation_data(results_df, essays_df)
    
    # Отправляем данные на валидацию
validation_result = validate_with_hackathon_service(validation_data)
    
    # Выводим метрики валидации
print_validation_metrics(validation_result)
    
    # Сохраняем результат валидации в файл
with open("validation_results.json", "w", encoding="utf-8") as f:
    json.dump(validation_result, f, ensure_ascii=False, indent=2)
print("💾 Результаты валидации сохранены в validation_results.json")
    
    # Дополнительно: сохраняем отправленные данные для отладки
with open("sent_validation_data.json", "w", encoding="utf-8") as f:
    json.dump(validation_data, f, ensure_ascii=False, indent=2)
print("💾 Отправленные данные сохранены в sent_validation_data.json")
print("\n📊 Логирование результатов в ClearML...")
    


Начинаем обработку 56 сочинений...
Сочинение 1: 96 слов
Обработано сочинение 1/56 (ID: 1)
Сочинение 2: 154 слов
Обработано сочинение 2/56 (ID: 2)
Сочинение 6: 142 слов
Обработано сочинение 3/56 (ID: 6)
Сочинение 7: 91 слов
Обработано сочинение 4/56 (ID: 7)
Сочинение 8: 179 слов
Обработано сочинение 5/56 (ID: 8)
Сочинение 9: 103 слов
Обработано сочинение 6/56 (ID: 9)
Сочинение 10: 85 слов
Обработано сочинение 7/56 (ID: 10)
Сочинение 11: 115 слов
Обработано сочинение 8/56 (ID: 11)
Сочинение 12: 180 слов
Обработано сочинение 9/56 (ID: 12)
Сочинение 13: 126 слов
Обработано сочинение 10/56 (ID: 13)
Сочинение 14: 156 слов
Обработано сочинение 11/56 (ID: 14)
Сочинение 15: 79 слов
Обработано сочинение 12/56 (ID: 15)
Сочинение 3: 85 слов
Обработано сочинение 13/56 (ID: 3)
Сочинение 4: 120 слов
Обработано сочинение 14/56 (ID: 4)
Сочинение 5: 110 слов
Обработано сочинение 15/56 (ID: 5)
Сочинение 41: 121 слов
Обработано сочинение 16/56 (ID: 41)
Сочинение 42: 102 слов
Обработано сочинение 17/56 (ID

In [41]:
test_word_count()

Текст: 'Привет, как дела?'
Слов: 3

Текст: 'М.Ю. Лермонтов - великий поэт.'
Слов: 3

Текст: 'Я прочитал 5 книг за лето.'
Слов: 5

Текст: 'Из-за дождя мы остались дома.'
Слов: 5

Текст: 'Несмотря на погоду, мы пошли гулять.'
Слов: 6



In [53]:
clearml_task = setup_clearml_task(
        task_name="Essay Evaluation Pipeline", 
        project_name="OGE_Essay_Scoring"
    )
model_info = save_model_artifacts(llm, {
        "chain1": "type1", 
        "chain2": "type2", 
        "chain3": "type3"
    })
    # Логируем метрики
print("\n📊 Логирование результатов в ClearML...")
    
    # Логируем метрики (включая score)
log_clearml_metrics(clearml_task, results_df, validation_result, model_info)
    
    # Логируем детали валидации (особенно score)
log_validation_details(clearml_task, validation_result, validation_data)
    
    # Логируем артефакты
artifacts_to_log = [
    "submission.csv",
    "объяснения_оценок.txt", 
    "validation_results.json",
    "sent_validation_data.json",
    "model_artifacts/model_info.json",
    "model_artifacts/prompt_texts.json"
    ]
log_clearml_artifacts(clearml_task, artifacts_to_log)
    
    # ✅ Завершаем задачу
if clearml_task:
    clearml_task.close()

Could not read Jupyter Notebook: No module named 'nbconvert'
Please install nbconvert using "pip install nbconvert"


ClearML Task: created new task id=c1a535289275423a880fa508c7537229
ClearML results page: https://app.clear.ml/projects/b3d51c5bcc7f4a5689073f065b54da5a/experiments/c1a535289275423a880fa508c7537229/output/log
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
✅ ClearML задача инициализирована с конфигурациями промтов
✅ Артефакты модели и промты сохранены в model_artifacts

📊 Логирование результатов в ClearML...
❌ Ошибка логирования в ClearML: 'Task' object has no attribute 'log_metrics'
✅ Детали валидации записаны в ClearML
✅ Артефакт submission.csv загружен в ClearML
✅ Артефакт объяснения_оценок.txt загружен в ClearML
✅ Артефакт validation_results.json загружен в ClearML
✅ Артефакт sent_validation_data.json загружен в ClearML
✅ Артефакт model_artifacts/model_info.json загружен в ClearML
✅ Артефакт model_artifacts/prompt_texts.json загружен в ClearML
